In [1]:
savevar(fn, x) = write(fn, string(x))
loadvar(fn) = read(fn, String) |> Meta.parse |> eval

loadvar (generic function with 1 method)

In [2]:
A = randn(ComplexF64, 4, 3, 2)

4×3×2 Array{ComplexF64, 3}:
[:, :, 1] =
  0.0362951+0.721473im  0.0837136+0.394688im   0.825512-1.41153im
   0.370305-0.282639im  -0.212329+0.713438im  -0.948288-0.66423im
 0.00378931+0.804708im   0.138855+0.239906im    1.60772-0.0895067im
  -0.339606+0.468061im  -0.436768-0.143496im   -1.54527+1.80814im

[:, :, 2] =
 -0.207455+0.272931im    0.31445-0.445149im  0.0136171-1.89719im
  0.738489-0.88716im    0.142878-1.1181im     0.544036+0.205605im
 -0.972507+1.23239im   -0.730273+0.660781im  -0.345665+0.197431im
  -0.21103+0.767076im   -0.41309-0.869451im  -0.396777-0.0188116im

In [3]:
savevar("tmp/A.txt", A)

1049

In [4]:
read("tmp/A.txt", String)

"[0.03629513214642978 + 0.7214732561208483im 0.08371355465181504 + 0.39468795761551956im 0.8255116799427007 - 1.411534139418578im; 0.37030479611095785 - 0.28263916307533404im -0.2123291731569978 + 0.7134384867868138im -0.9482880717326637 - 0.6642304489305854im; 0.00378930" ⋯ 508 bytes ⋯ "292im; -0.9725073828228493 + 1.2323930723733192im -0.7302732700063497 + 0.6607809770224146im -0.3456652244639833 + 0.19743079465439087im; -0.2110304977521085 + 0.7670761936407169im -0.41308972362962176 - 0.8694509938199181im -0.3967767365981693 - 0.018811645954688684im]"

In [5]:
A_load = loadvar("tmp/A.txt")

4×3×2 Array{ComplexF64, 3}:
[:, :, 1] =
  0.0362951+0.721473im  0.0837136+0.394688im   0.825512-1.41153im
   0.370305-0.282639im  -0.212329+0.713438im  -0.948288-0.66423im
 0.00378931+0.804708im   0.138855+0.239906im    1.60772-0.0895067im
  -0.339606+0.468061im  -0.436768-0.143496im   -1.54527+1.80814im

[:, :, 2] =
 -0.207455+0.272931im    0.31445-0.445149im  0.0136171-1.89719im
  0.738489-0.88716im    0.142878-1.1181im     0.544036+0.205605im
 -0.972507+1.23239im   -0.730273+0.660781im  -0.345665+0.197431im
  -0.21103+0.767076im   -0.41309-0.869451im  -0.396777-0.0188116im

In [6]:
A_load == A

true

In [7]:
B = ["Foo", "Bar", "Baz"]

3-element Vector{String}:
 "Foo"
 "Bar"
 "Baz"

In [8]:
savevar("tmp/B.txt", B)

21

In [9]:
read("tmp/B.txt", String)

"[\"Foo\", \"Bar\", \"Baz\"]"

In [10]:
B_load = loadvar("tmp/B.txt")

3-element Vector{String}:
 "Foo"
 "Bar"
 "Baz"

In [11]:
B_load == B

true

In [12]:
D = Dict(:A => A, :B => B)

Dict{Symbol, Array} with 2 entries:
  :A => [0.0362951+0.721473im 0.0837136+0.394688im 0.825512-1.41153im; 0.370305…
  :B => ["Foo", "Bar", "Baz"]

In [13]:
savevar("tmp/D.txt", D)

1105

In [14]:
read("tmp/D.txt", String)

"Dict{Symbol, Array}(:A => [0.03629513214642978 + 0.7214732561208483im 0.08371355465181504 + 0.39468795761551956im 0.8255116799427007 - 1.411534139418578im; 0.37030479611095785 - 0.28263916307533404im -0.2123291731569978 + 0.7134384867868138im -0.9482880717326637 - 0.6642" ⋯ 564 bytes ⋯ ".2323930723733192im -0.7302732700063497 + 0.6607809770224146im -0.3456652244639833 + 0.19743079465439087im; -0.2110304977521085 + 0.7670761936407169im -0.41308972362962176 - 0.8694509938199181im -0.3967767365981693 - 0.018811645954688684im], :B => [\"Foo\", \"Bar\", \"Baz\"])"

In [15]:
D_load = loadvar("tmp/D.txt")

Dict{Symbol, Array} with 2 entries:
  :A => [0.0362951+0.721473im 0.0837136+0.394688im 0.825512-1.41153im; 0.370305…
  :B => ["Foo", "Bar", "Baz"]

In [16]:
D_load == D

true

In [17]:
module O
struct Foo{A, B}
    a::A
    b::B
end
end

Main.O

In [18]:
foo = O.Foo(A, B)

Main.O.Foo{Array{ComplexF64, 3}, Vector{String}}([0.03629513214642978 + 0.7214732561208483im 0.08371355465181504 + 0.39468795761551956im 0.8255116799427007 - 1.411534139418578im; 0.37030479611095785 - 0.28263916307533404im -0.2123291731569978 + 0.7134384867868138im -0.9482880717326637 - 0.6642304489305854im; 0.0037893062964498773 + 0.8047078956204896im 0.13885534229280388 + 0.23990577877845826im 1.6077215336452866 - 0.08950669805809473im; -0.3396060871893301 + 0.4680605808371213im -0.4367680538509532 - 0.14349610351963074im -1.5452678726069151 + 1.8081375054930435im;;; -0.20745500431340289 + 0.27293133965547994im 0.3144500351426793 - 0.4451493437639256im 0.01361713843613112 - 1.8971857410744895im; 0.7384892674464592 - 0.8871601718961595im 0.14287797559065318 - 1.1181031306303615im 0.5440359643714784 + 0.2056051070328292im; -0.9725073828228493 + 1.2323930723733192im -0.7302732700063497 + 0.6607809770224146im -0.3456652244639833 + 0.19743079465439087im; -0.2110304977521085 + 0.7670761936

In [19]:
savevar("tmp/foo.txt", foo)

1122

In [20]:
read("tmp/foo.txt", String)

"Main.O.Foo{Array{ComplexF64, 3}, Vector{String}}([0.03629513214642978 + 0.7214732561208483im 0.08371355465181504 + 0.39468795761551956im 0.8255116799427007 - 1.411534139418578im; 0.37030479611095785 - 0.28263916307533404im -0.2123291731569978 + 0.7134384867868138im -0.94" ⋯ 581 bytes ⋯ "93 + 1.2323930723733192im -0.7302732700063497 + 0.6607809770224146im -0.3456652244639833 + 0.19743079465439087im; -0.2110304977521085 + 0.7670761936407169im -0.41308972362962176 - 0.8694509938199181im -0.3967767365981693 - 0.018811645954688684im], [\"Foo\", \"Bar\", \"Baz\"])"

In [21]:
foo_load = loadvar("tmp/foo.txt")

Main.O.Foo{Array{ComplexF64, 3}, Vector{String}}([0.03629513214642978 + 0.7214732561208483im 0.08371355465181504 + 0.39468795761551956im 0.8255116799427007 - 1.411534139418578im; 0.37030479611095785 - 0.28263916307533404im -0.2123291731569978 + 0.7134384867868138im -0.9482880717326637 - 0.6642304489305854im; 0.0037893062964498773 + 0.8047078956204896im 0.13885534229280388 + 0.23990577877845826im 1.6077215336452866 - 0.08950669805809473im; -0.3396060871893301 + 0.4680605808371213im -0.4367680538509532 - 0.14349610351963074im -1.5452678726069151 + 1.8081375054930435im;;; -0.20745500431340289 + 0.27293133965547994im 0.3144500351426793 - 0.4451493437639256im 0.01361713843613112 - 1.8971857410744895im; 0.7384892674464592 - 0.8871601718961595im 0.14287797559065318 - 1.1181031306303615im 0.5440359643714784 + 0.2056051070328292im; -0.9725073828228493 + 1.2323930723733192im -0.7302732700063497 + 0.6607809770224146im -0.3456652244639833 + 0.19743079465439087im; -0.2110304977521085 + 0.7670761936

In [22]:
(foo_load.a, foo_load.b) == (foo.a, foo.b)

true